In [ ]:
!pip install git+https://github.com/microsoft/dowhy.git

In [ ]:
import numpy as np
import pandas as pd
import dowhy

In [ ]:
df = pd.read_csv("bpi2017_enhancedData.csv")

In [ ]:
digraph {treatment->successful; CreditScore-> treatment; OfferedAmount-> successful; NumberOfTerms-> successful; MonthlyCost-> successful;}

In [ ]:
model=CausalModel(
        data = df,
        treatment=data['treatment_name'],
        outcome=data['outcome_name'],
        graph=data['gml_graph'
        )

In [ ]:
# Run a linear regression of column successful on treatment in df
import statsmodels.api as sm

X = df['treatment'].astype(str)
y = df['successful'].astype(int)

X = sm.add_constant(X)

ols = sm.OLS(y, X).fit()

# Display a more parsimonious results summary
print(ols.summary().tables[1])

In [ ]:
# Check whether causal effect is identified and return target estimands
identified_estimand = model.identify_effect()

In [ ]:
# Estimate the causal effect using inverse probability weighting
estimate = model.estimate_effect(identified_estimand, method_name="backdoor.propensity_score_weighting")

In [ ]:
# Check sensitivity of obtained estimate to unobserved confounders
refute_results = model.refute_estimate(identified_estimand, estimate, method_name="add_unobserved_common_cause")